# Lab 3: Gestión de Índices

Con esta practica pretendemos familiarizarnos con las principales operaciones de gestión que se pueden realizar sobre un índice, tanto desde la API de Elasticsearch como desde la propia interfaz de Kibana:

* Abrir/cerrar un índice
* Forzrar el merge de un índice
* Forzar el Refresh de un índice
* Forzar el Flush de un índice
* Crear políticas de ciclo de vida de un ídice


## 1. Gestión básica de un índice

Empezaremos viendo las operaciones de gestión más básicas sobre un índcie.

### 1.1. Cerrar y abrir índices

Cuando un ídice esta operativo y en un estado de funcionamiento normal, se dice que el índice está abierto y por tanto permite realizar todas las operaciones previstas sobre un índice, indexación de nuevos documentos, modificación y borrado de documetnos y por supuesto consulta y búsqueda de documentos.

Para realizar todas estas operaciones, Elasticsearch genra un conjunto de información que almacena en memoira asocida a la gestión de un índice, por ejemplo, los índices invertidos de los campos analizados, las caches de consulta, buffers de memoria para las operaciones, translog, segmentos no mergeados en disco, etc. 

Cuando un índice deja de tener un uso intensivo o deja de tener uso pero no queremos perder los documentos almacenados, entonces podemos cerrarlo, liberando todo ese hoverhead asociado a la gestión del índice y por tanto liberando recursos de los nodos del cluster. Conviene tener en cuenta que el espacio en disco que almacena los documetos se sigue usando para no perder información.

Un índice cerrado no permite operaciones de escritura ni lectura sobre él, ni tampoco operaciones de indexación o consulta de documentos.

#### Cerrar un índice

Para cerrar un índice desde la API de elasticsearch tendremos que realizar la siguiente petición:

`
POST /my-index-000001/_close
`

También es posible cerrar mas de un índice a la vez utilizando patrones en el nobre de los índices:

`
POST /my-index-*/_close
`

#### Abrir un índice

Realiza la operación inversa, reabre un índice cerraro y regenra los índices invertidos y reactiva la operativa sobre él. Esta opración tarda un tiempo en realizarse mientras reindexa todos los documentos del índcie.

Para abrir un índice usando la API de elasticsearch realizaremos la siguiente operación:
`
POST /my-index-000001/_open
`

Igual que con la operación de cierre, podemos abrir más de un índice a la vez utilizando patrones en sus nombres:

`
POST /my-index-*/_open
`

**Ejercicio 1:** 
* Crea un índice con 5 shards y una réplica.
* Cierra el índice usando la API.
* Consulta su estado en Kibana.
* Realiza alguna consulta sobre el índice.
* Reabre el índice.
* Consulta su estado en Kibana.
* Realiza alguna consulta sobre el índice.

### 1.2. Force Merge

Esta operación fuerza la fusión de los segmentos de un shard sin tener que esperar al proceso normal de elasticsearch.

Además permite liberar el espacio de los documentos borrados.

Para realizar esta operación a través de la API de elasticsearch realizaremos la siguiente petición:

`
POST /my-index-000001/_forcemerge
`

Se puede realizar esta operación sobre más de un índice utilizando patrones:

`
POST /my-index-*/_forcemerge
`

**Ejercicio 2:**
* Sobre el índice que creaste en el ejecicio 1 inserta al menos 10 documentos.
* Borra 3 documentos de los que acabas de insertar.
* Consulta en Kibana cuantos documentos marcados como borrados hay en los shards.
* Fuerza la fusión de los segmentos.
* Vuelve a consultar en Kibana los documentod marcados como borrado. ¿Cuántos hay ahora respescto a la consutlta anterior?

### 1.3. Refresh

Cuando se realiza la inserción de un documento, desde que se raliza la operación hasta que el documento se puede consultar pasa como mínimo 1 segundo que es el tiempo que pasa entre procesos de refresco. El refresco toma las operaciones que se ncuentran encoladas en el buffer en memoria y las persiste en un segmento de Lucene en la cache de disco. Sólo cuando el documento se persiste en un segmento está disponible para su búsqueda.

Para realizar esta operación a través de la API de elasticsearch utilizaremos la siguiente petición:

`
POST /my-index-000001/_refresh
`

También podemos realizar esta operación sobre varios índices utilizando patrones:

`
POST /my-index-*/_refresh
`

**Ejercicio 3:**
* Fuerza el refresh del índice que create en el ejercicio 1.

### 1.4. Flush

El proceso de flush fuerza a que todas las operaciones que se encuentran en el transaction log se realicen y se almacenen de forma definitiva en el índice de Lucene en disco sin esperar al proceso de commit. 

Para realizar esta operación a través de la API de elasticsearch realizaremos la siguiente petición:

`
POST /my-index-000001/_flush
`

Como en las operaciones anteriores, podemos realizar esta operación sobre más de un índice a la vez utilizando patrones:

`
POST /my-index-*/_flush
`

**Ejercicio 4:**
* Fuerza el flush del índice que create en el ejercicio 1.

### 1.5. Cómo realizar estas operaciones en Kibana

Todas estas operaciones las podemos realizar a través de Kibana. Para ello vamos a entrar en la sección de "Index Management".

Al seleccionar uno de los índices aparecerá una ventana emergente con la información sobre el índice seleccionado.
Abajo a la derecha puede ver un botón "^Manage" que al clicarlo aparece un menú emergente con todas las opciones que hemos visto hasta ahora.

<img src="../images/els/gestionIndices.png" alt="index management"/>

**Ejercicio 5:**
* Prueba a realizar al menos uno de los ejercicios anteriores usando la interfaz de Kibana.

## 2. Políticas de gestión de ciclo de vida de un índice.

Puesto que la creación de una política para gestionar el ciclo de vida de un índice requiere realizar una serie de operaciones de forma conjunta lo vamos a ver en detalle y paso a paso.

Definir una buena política para gestión de la vida de un índice requiere entender y las necesidades del caso de uso para el que se están utilizando los índices para adecuarse de forma correcta a sus necesidades. 

Para realizar esta práctica vamos a suponer que la información que estamos almacenando proviene de mediciones de sensores de las diferentes máquinas de una linea de producción de una fabrica para posteriormente, realizar mantenimiento predictivo de los diferentes activos de esta línea de producción entrenando modelos de IA.

Teniendo este caso de uso en mente, lo primero que tenemos que analizar es la naturaleza de nuestros datos. En este caso, los datos que vamos a recibir responden a mediciones temporales de los diferentes sensores con su respectiva metrica y su valor asociados a la marca de tiempo en la que se guenero dicha medición. Por tanto responde a una serie temporal de datos, donde el orden de producción de la información es importante, para poder analizar la correlación entre los datos y la causalidad que pueda haber entre mediciones. 

El segundo punto a tener en cuenta, es el tiempo de validez de nuestra analítica, es decir el tiempo en el que el dato inferido (resultado de los modelos) tiene validez. Por ejemplo, si nuestro modelo tiene que identificar un modo de fallo de un motor para poder actura inmediatamente parándolo para evitar la rotura o deterioro de los componentes conectados a él, el tiempo de validez del resultado inferido (el modo de fallo) es inferior al segundo, puesto que una vez se ha producico el tiempo de respuesta del sistema está en ese rango de tiempo de actuación.

Por otro lado, si lo que queremos hacer es un mantenimiento predictivo, es decir predcir cuál es el tiempo de vida restante de un componente de un activo para alargar al máximo el tiempo de vida del componente y programar su sustitución sin afectar a la cadena de producción. En este caso el tiempo de validez de la inferencia puede ser de días o incluso semanas.

Todos estos casos de uso nos indicarán la disponibiliad y validez de dato:

<img src="../images/els/tiempoDeVidaDato.png" alt="index management"/>


Otro aspecto a tener en cuenta es el volumen de datos, según se van añadiendo más sensores, el volumen de datos a ingestar y retener aumenta significativamente y por tanto el coste de almacenamiento. Por lo que es necesario adecuar la disponibilidad y el tiempo de retención del dato a las necesidades del caso de uso. 

El dato mas disponible es el más caro, por lo que no interesa sólo tener el volumen de datos necesario y durante el tiempo de validez del dato para la analítica del caso de uso, e ir rotando a niveles de acceso mas económicos hasta definir cual es su momento de archivado y posterior borrado (borrar mucahs veces significa pasar el dato a otro tipo de almacenamiento en otro data store pensado para almacenar grandes volúmenes de datos a muy bajo coste, pero con una latencia de acceso al dato de incluso días).

Elasticserch nos permite definir "Data Tiers". Cada Data Tier es un conjunto de nodos que tienen unas características comunes en cuanto a hardware. Esto es muy útil por ejemplo, cuando se instala Elasticsearch en un proveedor de infraestructura en la nuve, pudiendo definir nodos mas caros o mas baratos en función de su hardware, asignarlos a un tier, para posteriormente alojar los índices de cada nivel del ciclo de vida en el tier adecuado según sus necesidades. De esta forma podemos controlar el coste de la infraestructura.

<img src="../images/els/DataTiers.png" alt="index management"/>

Elasticsearch nos permite por un lado identificar y definir diferentes índices para cada uno de los niveles del dato, un mecanismo de rotación de datos entre índices basado en tiempo y/o en volumen de datos almacenado y un poceso de archivado y borrado. Todo esto se orquesta y automatiza definiendo una política de gestión del ciclo de vida de un índice. 


### 2.1. Definición de la política

Para definir la política, lo primero que tenmos que hacer es definir los niveles del ciclo de vida de un índice. Elasticserach define 5 niveles:

* **Hot:** El nivel por defecto y por lo tanto obligatorio y se utiliza para alojar los datos más actuales. Se pueden añadir, borrar, modificar y consultar documentos.
* **Warm:** El índice es de sólo lectura pero se puede seguir consultando. Contiene documentos que no se suelen modificar, pero que se necesita seguir consultando.
* **Cold:** El índice es de sólo lectura y aloja documentos que no se suelen consultar con mucha frecuencia.
* **Frozen:** Igual que el nivel anterior, pero la frecuencia de acceso es aún menor y el tiempo de acceso a los documentos es mayor (menor disponibilidad del dato).
* **Delete:** El índice se borra ya que los documentos se consideran que no tienen validez o no son útiles.

En Elasticsearch la unidad mínima de gestión de datos es el índice, por lo que cada uno de estos niveles estará compuesto por uno o varios índices.

Y una vez identificados, es necesario definir las acciones que se tienen que producir para pasar los datos de un nivel a otro. Para pasar los datos de un nivel a otro lo que se realiza es una poreación de rotación de indíces o rollover.

Vamos a ver los diferentes conceptos que nos proporciona Elasticaserch para construir una política en base a lo definido previamente:

* **Bootstrap index**: Es el índice inicial que hay que crear manualmente y en el que se empezarán a añadir los datos de nuestro caso de uso. Constituirá el primer índice de escritura y por tanto el primer índice del nivel Hot.
* **Write alias**: Los clientes de Elasticsearch no tienen por que saber cuál es el índice activo o sobre el que pueden añadir mas datos. Para interactuar con el índice de escrutura activo actual se utiliza un alias de índice, que es el nombre de índice que utilizarán los clientes y que la política de gestión se encargará de asignar a los nuevos índices de escritura.
* **Index rollover**: Un nuevo índcie de escritura es creado y se le asigna el alias de escritura. Al anerior índice se le asigna permisos de sólo lectura. Esta operación se realiza cuando el índice actual de escritura alcanza un tamaño máximo de datos o de documentos especificado o alcanza su edad máxima, es decir el tiempo máximo que puede estar activo un índice para escritura.

La escritura siempre se realiza sobre el índice activo de escritura a través del alias y la lectura se puede ejecutar sobre dodos los índices usando un wildcard:

<img src="../images/els/ilmEscrituraLectura.png" alt="index management"/>

### 2.2. Implementación de la política

Vamos a ver los pasos a ejecutar para definir la política a través de la API de Elasticsearch:

### Paso 1: Definir los niveles y las condiciones para el rollover entre niveles:

Nota: No todos los niveles y opciones están disponibles en la licencia gratuita. Para esta práctica sólo vamos a utilizar los niveles disponibles en la licencia gratuita.

Para definir la política realizaremos la apetición _ilm/policy/policy_name: 

`
PUT _ilm/policy/sensor-metrics-policy
{
    "policy": {
        "phases": {
            "hot": {
                "actions": {
                    "rollover": {
                        "max_age": "30d",
                        "max_size": "50gb"
                    }
                }
            },
            "warm": {
                "min_age": "1d",
                "actions": {}
            },
            "delete": {
                "min_age": "10d",
                "actions": {}
            }
        }
    }
}
`

Esta política ejecuta las siguientes acciones:
* El índice del nivel "hot" se rotará cuando alcance un tamaño de 50gb o tenga una antiguedad de 30 días, lo que ocurra primero. 
* Una vez pasado un día desde que el índice se rotó en "hot", el índcie hot se pasará al nivel "warm".
* Una vez pasado 10 días desde que el índice entró en el nivel "warm", se borrará del cluster.

En cada nivel se pueden definir diferentes acciones:
* **allocate**: permite indicar en que nodos alojar las particiones del índice así como cambira el número de particiones y réplicas del índice. No se puede usar en el nivel "hot".
* **force_merge**: indica el número de segmentos que queremos que tenga el índice al ser rotado. Es necesario definir la acción de rollover. Convierte el índice en sólo lecutra. Valido para las fases "hot" y "warm".
* **migrate**: Mueve los shards del índcie al data tier especificado.

#### Ejercicio 6: 
Crea una política para nuestro caso de uso de sensores, en donde se cumplan las siguientes condiciones:
* Puesto que el volumen de datos que vamos almacenar provineiente de sensores que realizan mediciones cada milisegundo, el nivel hot se rotará cada h o cuando alcance un volumen de 1 GB.
* El dato permanecerá en el nivel "hot" durante un día ya que sólo queremos el dato para hacer inferencia de modelos.
* Para realizar entrenamiento de modelos el dato permanecera en el nivel "warm" durante 1 mes.
* Pasado el mes el dato se borrará.

Nota: aquí puedes consultar las unidades de tiempo: https://www.elastic.co/guide/en/elasticsearch/reference/8.4/api-conventions.html#time-units
y en este otro enlace las de volumen de datos: https://www.elastic.co/guide/en/elasticsearch/reference/8.4/api-conventions.html#byte-units

### Paso 2: Crear el índex template para los nuevos índices

Puesto que cada vez que se rota el índice activo de escritura se crea un nuevo índice, es necesario crear un index template para crearlo con el esquema necesario. 
Para simplificar, y puesto que ya hemos visto como crear templates, sólo vamos a definir los campos necesarios para la creación de politicas.

`
PUT _index_template/web-logs
{
    "index_patterns": ["ilm-sensor-metrics-*"],
    "template": {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 1,
            "index.lifecycle.name": "sensor-metrics-policy",
            "index.lifecycle.rollover_alias": "ilm-sensor-metrics"
        }
    }
}
`

Estamos añadiendo dos settings nuevos:
* **index.lifecycle.name**: El nombre de la política que acabmos de definir, para saber con que patrón es necesario crear los nuevos índcies al ser rotados.
* **index.lifecycle.rollover_alias**: El alias de escritura que se le asignará al nuevo índice creado después del rotado.

#### Ejercicio 7:
Crea un index template para la política que estás difiniendo. Al crea el mapping, ten en cuenta los parámetros que quermos almacenar por medición:

* asset: Activo al que pertenece la medición, por ejemplo "motor 1" o "cabina de rodillos 23". Lo utilizaremos para filtrar las mediciones de un determinado activo.
* @timestamp: marca de tiempo en la que se realizó la medición.
* metric: metrica que se ha medido, por ejemplo "temperature" o "humidity". Se tuilizará para filtrar y hacer agregaciones.
* value: el valor de la medición de la métrica, por ejemplo -0.5.

### Paso 3: Creamos el bootstrap index

Como hemos visto antes, el índice inicial es necesario crearlo a mano.
Para simplificar, y puesto que ya hemos visto como crear índices, sólo vamos a definir los campos necesarios para la creación de politicas.

`
PUT ilm-sensor-metrics-00001
{
    "aliases": {
        "ilm-sensor-metrics":{
            "is_write_index": true
        }
    }
}
`

* El nombre del índice tiene que seguir el siguiente patrón `^.*-\d+$`es decir que termine en dígito. De esta manera la política sabrá como incrementar el numero de índice de forma automática.
* Es necesario asignarle el álias de escritura y a la vez indicarle que el índcie es de escritura puesto que va a ser el primer índice de escritura activo.

#### Ejercicio 8: 
Crea el bootstap index.

### Paso 4: Utilizar el índice con normalidad

Para la escritura utilizaremos siempre el álias:

`
POST ilm-sensor-metrics/_doc
{
    "asset": "motor 123",
    "timestamp": "2021-02-14T13:00:00.000Z",
    "metric": "temperature",
    "value": 24.5
}
`

y para la consulta lo realizaremos sobre todos los índices usando un wildcard:

`
GET ilm-sensor-metrics*/_search
`

Por último, siempre podemos forzar un rollover:

`
POST ilm-sensor-metrics/_rollover
`


#### Ejercicio 9:

* Inserta al menos 10 mediciones en el índcie.
* Fuerza un rollover.
* Consulta el número de índices que alojan el conjunto de datos del caso de uso.
* Inserta al menos 10 medicines más.
* Realiza una consulta que devuelva todas las mediciones insertadas.